1. Configuración del Entorno

In [ ]:
import ee
ee.Authenticate()
ee.Initialize()


Definir el Área de Interés (AOI) y el Rango de Fechas

In [ ]:
import geemap

# Crear un mapa centrado en Montería, Colombia
Map = geemap.Map(center=[8.75, -75.88], zoom=10)

# Definir el área de interés (AOI) como un rectángulo alrededor de Montería
aoi = ee.Geometry.Rectangle([-76.0, 8.6, -75.7, 8.9])

# Definir el rango de fechas
start_date = '2025-01-01'
end_date = '2025-06-01'


3. Cargar y Filtrar la Colección de Imágenes Sentinel-1

In [ ]:
# Cargar la colección de imágenes Sentinel-1
sentinel1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate(start_date, end_date) \
    .filter(ee.Filter.eq('instrumentMode', 'IW')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH')) \
    .filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))  # Órbita descendente


4. Calcular la Media Temporal de las Polarizaciones VV y VH

In [ ]:
# Calcular la media temporal para VV y VH
vv = sentinel1.select('VV').mean().clip(aoi)
vh = sentinel1.select('VH').mean().clip(aoi)


5. Calcular el Índice de Retrodispersión Cruzada (CR)

In [ ]:
# Calcular el índice CR (Cross Ratio)
cr = vv.subtract(vh).rename('CR')


6. Visualizar los resultados

In [ ]:
# Definir los parámetros de visualización
vv_vis = {'min': -20, 'max': 0}
vh_vis = {'min': -25, 'max': -5}
cr_vis = {'min': -5, 'max': 5}

# Agregar las capas al mapa
Map.addLayer(vv, vv_vis, 'VV')
Map.addLayer(vh, vh_vis, 'VH')
Map.addLayer(cr, cr_vis, 'CR')

# Mostrar el mapa
Map



7. Exportar los resultados

In [ ]:
# Exportar la imagen CR a Google Drive
task = ee.batch.Export.image.toDrive(
    image=cr,
    description='CR_Monteria',
    folder='GEE_Exports',
    fileNamePrefix='CR_Monteria',
    region=aoi,
    scale=10,
    crs='EPSG:4326'
)
task.start()